In [1]:
# lib's base 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# lib's de extração de dados
from pytrends.request import TrendReq
import snscrape.modules.twitter as sntwitter

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## 1. extração de dados

### 1.1. dados do twitter

In [2]:
%%time

###### IBOV ######

lista_tweets_01 = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper('IBOV since:2021-01-01 until:2021-12-31').get_items()):
    lista_tweets_01.append([tweet.date])
    
df_tweets_01 = pd.DataFrame(lista_tweets_01, columns=['DATA'])

###### IBOVESPA ######

lista_tweets_02 = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper('IBOVESPA since:2021-01-01 until:2021-12-31').get_items()):
    lista_tweets_02.append([tweet.date])
    
df_tweets_02 = pd.DataFrame(lista_tweets_02, columns=['DATA'])

###### BOVA11 ######

lista_tweets_03 = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper('BOVA11 since:2021-01-01 until:2021-12-31').get_items()):
    lista_tweets_03.append([tweet.date])
    
df_tweets_03 = pd.DataFrame(lista_tweets_03, columns=['DATA'])

Wall time: 1h 38min 14s


In [3]:
# transformar bases em séries temporais e contar citações por dia

df_tweets_01['DATA'] = df_tweets_01['DATA'].dt.date
df_tweets_01['DATA'] = pd.to_datetime(df_tweets_01['DATA'], format='%Y/%m/%d')
df_tweets_011 = df_tweets_01.groupby(['DATA']).size().reset_index(name = 'IBOV_TWITTER')

df_tweets_02['DATA'] = df_tweets_02['DATA'].dt.date
df_tweets_02['DATA'] = pd.to_datetime(df_tweets_02['DATA'], format='%Y/%m/%d')
df_tweets_022 = df_tweets_02.groupby(['DATA']).size().reset_index(name = 'IBOVESPA_TWITTER')

df_tweets_03['DATA'] = df_tweets_03['DATA'].dt.date
df_tweets_03['DATA'] = pd.to_datetime(df_tweets_03['DATA'], format='%Y/%m/%d')
df_tweets_033 = df_tweets_03.groupby(['DATA']).size().reset_index(name = 'BOVA11_TWITTER')

In [4]:
# merge mantendo somente dias em comum

df_twitter = pd.merge(df_tweets_011, df_tweets_022, how = 'inner', on = ['DATA'])
df_twitter = pd.merge(df_twitter, df_tweets_033, how = 'inner', on = ['DATA'])

df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348 entries, 0 to 347
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATA              348 non-null    datetime64[ns]
 1   IBOV_TWITTER      348 non-null    int64         
 2   IBOVESPA_TWITTER  348 non-null    int64         
 3   BOVA11_TWITTER    348 non-null    int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 13.6 KB


In [5]:
df_twitter.to_csv('../dados/outputs/df_twitter.csv')

### 1.2. dados do google

In [6]:
google_trends_01 = pd.read_csv('../dados/google_trends/trends_202101_202106.csv', skiprows=1)
google_trends_02 = pd.read_csv('../dados/google_trends/trends_202107_202112.csv', skiprows=1)

# unificação das bases

df_google_trends = pd.concat([google_trends_01, google_trends_02])
df_google_trends.rename(columns = {'IBOVESPA: (Brasil)':'IBOVESPA_GOOGLE', 'Dia':'DATA'}, inplace = True)
df_google_trends['DATA'] = pd.to_datetime(df_google_trends['DATA'])

df_google_trends.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 0 to 183
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DATA             365 non-null    datetime64[ns]
 1   IBOVESPA_GOOGLE  365 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 8.6 KB


In [7]:
df_google_trends.to_csv('../dados/outputs/df_google_trends.csv')

### 1.3. dados da b3

In [8]:
# leitura do arquivo - dados diários de 2021
arquivo_b3 = '../dados/b3/COTAHIST_A2021.txt'

# com base no layout disponibilizado pela B3
tamanho_campos = [2 ,8, 2, 12, 3, 12, 10, 3, 4, 13, 13, 13, 13, 13, 13, 13, 5, 18, 18, 13, 1, 8, 7, 13, 12, 3]

# dataframe pandas
dados_b3 = pd.read_fwf(arquivo_b3, widths = tamanho_campos)

In [9]:
# nomear colunas
dados_b3.columns = ['tipo_registro', 'data_pregao', 'cod_bdi', 'ticker', 'tipo_mercado', 'empresa', 
                    'especificacao_papel', 'prazo_dias_merc_termo', 'moeda_referencia', 'preco_abertura', 'preco_max', 
                    'preco_min', 'preco_medio', 'preco_ultimo_negocio', 'preco_melhor_oferta_compra', 
                    'preco_melhor_oferta_venda', 'qtd_negocios', 'qtd_papeis_negociados', 'vol_total_negociado', 
                    'preco_exercicio', 'ìndicador_correcao_precos', 'data_vencimento' , 'fator_cotacao', 
                    'preco_exercicio_pontos', 'cod_isin', 'num_distribuicao_papel']

# eliminar trailer do arquivo
linha = len(dados_b3['data_pregao'])
dados_b3 = dados_b3.drop(linha-1)

# alterar tipo da data
dados_b3['data_pregao'] = pd.to_datetime(dados_b3['data_pregao'], format='%Y/%m/%d')
dados_b3.set_index('data_pregao', inplace = True)

# ajustar valores para decimal
lista_dec = ['preco_abertura', 'preco_max', 'preco_min', 'preco_medio', 'preco_ultimo_negocio', 'preco_melhor_oferta_compra', 
           'preco_melhor_oferta_venda', 'vol_total_negociado', 'preco_exercicio', 'preco_exercicio_pontos']

for coluna in lista_dec:
    dados_b3[coluna]=[i/100. for i in dados_b3[coluna]]

In [10]:
# somente BOVA11
ticker_aux = dados_b3.loc[dados_b3['ticker'] == 'BOVA11']
print(ticker_aux.shape)

# somente as colunas "necessárias"
dados_b3 = ticker_aux[['ticker', 'empresa', 'qtd_negocios', 'qtd_papeis_negociados', 'vol_total_negociado']]
dados_b3.reset_index(inplace=True)
dados_b3.rename(columns = {'data_pregao':'DATA', 'ticker':'TICKER', 'empresa':'EMPRESA', 'qtd_negocios':'QTD_NEGOCIOS', 
                           'qtd_papeis_negociados':'QTD_PAPEIS_NEGOCIADOS', 'vol_total_negociado':'VOL_TOTAL_NEGOCIADO'}, 
                inplace = True)

(247, 25)


In [11]:
dados_b3.to_csv('../dados/outputs/dados_b3.csv')

## 2. construção do dataframe único e transformação em série temporal

### 2.1. unificar df's

In [12]:
df_01 = pd.merge(dados_b3, df_google_trends, how='inner', on = ['DATA'])
df_01 = pd.merge(df_01, df_twitter, how='inner', on = ['DATA'])

df_01 = df_01.sort_values('DATA')
print('quantidade de linhas/dias:', df_01.shape[0])
df_01.info()

quantidade de linhas/dias: 243
<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 2 to 242
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   DATA                   243 non-null    datetime64[ns]
 1   TICKER                 243 non-null    object        
 2   EMPRESA                243 non-null    object        
 3   QTD_NEGOCIOS           243 non-null    float64       
 4   QTD_PAPEIS_NEGOCIADOS  243 non-null    float64       
 5   VOL_TOTAL_NEGOCIADO    243 non-null    float64       
 6   IBOVESPA_GOOGLE        243 non-null    int64         
 7   IBOV_TWITTER           243 non-null    int64         
 8   IBOVESPA_TWITTER       243 non-null    int64         
 9   BOVA11_TWITTER         243 non-null    int64         
dtypes: datetime64[ns](1), float64(3), int64(4), object(2)
memory usage: 20.9+ KB


### 2.2. transformar data em índice

In [13]:
df_01.set_index('DATA', inplace=True)
df_01.head()

,TICKER,EMPRESA,QTD_NEGOCIOS,QTD_PAPEIS_NEGOCIADOS,VOL_TOTAL_NEGOCIADO,IBOVESPA_GOOGLE,IBOV_TWITTER,IBOVESPA_TWITTER,BOVA11_TWITTER
DATA,,,,,,,,,
2021-01-04,BOVA11,ISHARES BOVA,41278.0,8493210.0,9.730709e+08,69,83,298,6
2021-01-05,BOVA11,ISHARES BOVA,2390.0,9620560.0,1.094933e+09,77,128,277,7
2021-01-06,BOVA11,ISHARES BOVA,666.0,9094230.0,1.049862e+09,75,104,287,7
2021-01-07,BOVA11,ISHARES BOVA,50022.0,9250540.0,1.081833e+09,79,182,375,14
2021-01-08,BOVA11,ISHARES BOVA,42698.0,11955320.0,1.427557e+09,72,224,381,18


In [14]:
df_01.to_csv('../dados/outputs/st_variaveis.csv')